In [23]:
### Used only to run on Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

# Change de path to your drive
base_path = "gdrive/MyDrive/Colab_Notebooks/SelecaoUnicamp/data/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# !pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import math
from collections import defaultdict
from tqdm.notebook import tqdm
import nltk
from rank_bm25 import BM25Okapi
import numpy as np

In [25]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
stemmer = nltk.stem.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')

In [10]:
def preprocess_string(txt, remove_stop=True, do_stem=True, to_lower=True):
    """
    Return a preprocessed tokenized text.

    Args:
        txt (str): original text to process
        remove_stop (boolean): to remove or not stop words (common words)
        do_stem (boolean): to do or not stemming (suffixes and prefixes removal)
        to_lower (boolean): remove or not capital letters.

    Returns:
        Return a preprocessed tokenized text.
    """
    if to_lower:
        txt = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt)

    if remove_stop:
        tokens = [tk for tk in tokens if tk not in stop_words]
    if do_stem:
        tokens = [stemmer.stem(tk) for tk in tokens]
    return tokens

In [12]:
def create_index(documents):
    """
    Create the index of documents
    Args:
        documents: List of documents.

    Returns:
        indexes (defaultdict):
            Dictionary with the indexes of documents
    """
    indexes = defaultdict(list)
    for doc_id, document in documents.items():
        for term in document:
            indexes[term].append(doc_id)
    return indexes

In [13]:
def compute_idf_weights(indexes, num_docs):
    """
    Compute the IDF weights for each term
    Args:
        indexes: List of indexes
        num_docs: Number of documents

    Returns:
        idf_weights (dict):
            List of IDF weights
    """
    idf_weights = {}
    for term, doc_ids in indexes.items():
        idf_weights[term] = math.log(num_docs / len(doc_ids))
    return idf_weights

In [14]:
def compute_bm25_score(queries, documents, indexes, idf_weights, k1=1.5, b=0.75):
    """
    Compute the BM25 score for a query and a document
    Args:
        queries: List of queries
        documents: List of documents
        indexes: List of indexes
        idf_weights: List of IDF weights
        k1: Control the impact of term frequency on the score.
        b: Control the impact of document length on the score

    Returns:

    """
    score = 0
    doc_len = len(documents)
    avg_doc_len = sum(map(len, indexes.values())) / len(indexes)
    for term in queries:
        if term not in indexes:
            continue
        tf = documents.count(term)
        idf = idf_weights[term]
        numerator = tf * (k1 + 1)
        denominator = tf + k1 * (1 - b + b * (doc_len / avg_doc_len))
        score += idf * (numerator / denominator)
    return score

In [15]:
def rank_documents(queries, documents, indexes, idf_weights):
    """
    Rank the documents for a query using BM25
    Args:
        queries: List of queries
        documents: List of documents
        indexes: List of indexes
        idf_weights: List of IDF weights

    Returns:
        scores (dict):
            Dictionary with the scores of documents
    """
    scores = {}
    for doc_id, document in documents.items():
        score = compute_bm25_score(queries, document, indexes, idf_weights)
        scores[doc_id] = score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [16]:
def prepare_data():
    """
    Load the documents, queries and relevance from the CISI collection

    Returns:

    """
    documents: dict[int, list] = {}
    doc_id = ""
    doc_text = ""
    lines = read_file("CISI.ALL")

    for line in lines:
        if line.startswith(".I"):
            doc_id = int(line.split(" ")[1].strip()) - 1
        elif line.startswith(".X"):
            documents[doc_id] = preprocess_string(doc_text)

            doc_id = ""
            doc_text = ""
        else:
            doc_text += line.strip()[3:] + " "

    lines = read_file("CISI.QRY")

    queries: dict[int, list] = {}
    qry_id = ""
    for line in lines:
        if line.startswith(".I"):
            qry_id = int(line.split(" ")[1].strip()) - 1
        elif line.startswith(".W"):
            queries[qry_id] = preprocess_string(line.strip()[3:])
            qry_id = ""

    relevance: dict[int, list] = {}
    with open(f'{base_path}CISI.REL') as f:
        for line in f.readlines():
            qry_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) - 1
            doc_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]) - 1
            if qry_id in relevance:
                relevance[qry_id].append(doc_id)
            else:
                relevance[qry_id] = []
                relevance[qry_id].append(doc_id)

    return documents, queries, relevance

In [17]:
def read_file(file_name):
    """
    Read file and return the lines of file
    Args:
        file_name: Name of file

    Returns:
        lines (str):
            Lines of file
    """
    with open(base_path + file_name) as f:
        lines = ""
        for line in f.readlines():
            lines += "\n" + line.strip() if line.startswith(".") else " " + line.strip()
        lines = lines.lstrip("\n").split("\n")
    return lines

In [18]:
def evaluate_system(documents, queries, indexes, idf_weights, relevance):
    """
    Evaluate the system using the relevance judgments
    Args:
        documents: List of documents
        queries: List of queries
        indexes: List of indexes
        idf_weights: List of IDF weights
        relevance: List of relevances

    Returns:

    """
    num_queries = 0
    total_precision = 0
    total_recall = 0
    for query_id, query in tqdm(queries.items()):
        if query_id in relevance.keys():
            num_queries += 1
            top_docs = rank_documents(query, documents, indexes, idf_weights)[:10]
            relevant_docs = relevance[query_id]
            num_retrieved = len(top_docs)
            num_relevant = len(relevant_docs)
            num_common = len(set(relevant_docs).intersection(set(d[0] for d in top_docs)))
            precision = num_common / num_retrieved if num_retrieved > 0 else 0
            recall = num_common / num_relevant if num_relevant > 0 else 0
            total_precision += precision
            total_recall += recall

    mean_precision = total_precision / num_queries
    mean_recall = total_recall / num_queries
    f1_score = 2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0

    return mean_precision, mean_recall, f1_score

In [19]:
def mean_reciprocal_rank(bool_results, k=10):
    """
    Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

In [20]:
def results_from_query(qry_id, bm25):
    """
    Calculate the results for each query.
    Args:
        qry_id: Query id
        bm25: Instance of object to bm25

    Returns:
        Relevance results
    """
    query = queries[qry_id]
    rel_docs = []
    if qry_id in relevance:
        rel_docs = relevance[qry_id]
    scores = bm25.get_scores(query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    return sorted_masked_relevance_results

In [28]:
documents, queries, relevance = prepare_data()
indexes = create_index(documents)
idf_weights = compute_idf_weights(indexes, len(documents))

mean_precision, mean_recall, f1_score = evaluate_system(documents, queries, indexes, idf_weights, relevance)
print(mean_precision)
print(mean_recall)
print(f1_score)

  0%|          | 0/112 [00:00<?, ?it/s]


***************

0.2276315789473683
0.0929330429055127
0.13198271955708177


In [29]:
bm25 = BM25Okapi(documents.values())
results = [results_from_query(qry_id, bm25) for qry_id in list(queries.keys())]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.4187
